# deploying yaml on optimized python images

* one node
* lightgbm
* 10 mio samples / 20 features
* code stored as yaml in github
* precomiled images using optimized for cpu python libraries 
    * **[yjbds/mlrun-ds](https://hub.docker.com/repository/docker/yjbds/mlrun-ds)** a data science stack
    * **[yjbds/mlrun-files](https://hub.docker.com/repository/docker/yjbds/mlrun-files)** a parquet/pandas stack

## imports

In [1]:
import mlrun
import os
import numpy as np
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters

In [2]:
CODE_BASE   = '/User/repos/functions/'           
TARGET_DATA_PATH   = '/User/mlrun/models'

SKLEARN_CLASSIFIER = 'lightgbm.sklearn.LGBMClassifier'
MODEL_KEY          = 'model'
MODEL_NAME         = 'lgb-classifier.pkl'
VERBOSE            = False

_____
## train a classifier

In [3]:
testfn = mlrun.code_to_function(
    kind='job', 
    filename=os.path.join(CODE_BASE, 'train', 'sklearn-classifier.py'))
testfn.build_config(base_image='yjbds/mlrun-ds:latest', commands=[])
testfn.export(os.path.join(CODE_BASE, 'train', 'sklearn-classifier.yaml'))

[mlrun] 2020-01-26 19:17:00,914 function spec saved to path: /User/repos/functions/train/sklearn-classifier.yaml


In [4]:
trainfn = mlrun.import_function(
    os.path.join(CODE_BASE+'train/sklearn-classifier.yaml')
).apply(mlrun.mount_v3io())

In [5]:
trainfn.deploy(skip_deployed=True, with_mlrun=False)

[mlrun] 2020-01-26 19:17:15,462 starting remote build, image: .mlrun/func-default-sklearn-classifier-latest
INFO[0000] Resolved base name yjbds/mlrun-ds:latest to yjbds/mlrun-ds:latest 
INFO[0000] Resolved base name yjbds/mlrun-ds:latest to yjbds/mlrun-ds:latest 
INFO[0000] Downloading base image yjbds/mlrun-ds:latest 
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:d7724d11d33770dd3f65bee87ce7bf9f182428e96d53343f82ab5fce506f875b: no such file or directory 
INFO[0000] Downloading base image yjbds/mlrun-ds:latest 
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Downloading base image yjbds/mlrun-ds:latest 
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:d7724d11d33770dd3f65bee87ce7bf9f182428e96d53343f82ab5fce506f875b: no such file or directory 
INFO[0000] Downloading base image yjbds/mlrun-ds:latest 
INFO[0000] Unpacking rootfs as cmd RUN pip install mlrun requires it. 
INFO[0048] Tak

True

In [7]:
task = mlrun.NewTask()
task.with_params(
    SKClassifier=SKLEARN_CLASSIFIER,
    callbacks = [],
    xtrain='/User/mlrun/models/xtrain.pqt',
    ytrain='/User/mlrun/models/ytrain.pqt',
    xvalid='/User/mlrun/models/xvalid.pqt',
    yvalid='/User/mlrun/models/yvalid.pqt',
    target_path='/User/mlrun/models',
    name=MODEL_NAME,
    key=MODEL_KEY,
    verbose=VERBOSE)

In [8]:
tsk = trainfn.run(task, handler='train')

[mlrun] 2020-01-26 19:20:00,276 starting run train uid=984d1c76d3744b8593395d4cec4c06e7  -> http://mlrun-api:8080
[mlrun] 2020-01-26 19:20:00,370 Job is running in the background, pod: train-7mmh2
[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[LightGBM] [Warning] Cannot change bin_construct_sample_cnt after constructed Dataset handle.
[mlrun] 2020-01-26 19:21:48,490 log artifact training-validation-plot.html at training-validation-plot.html, size: 36420, db: Y
[mlrun] 2020-01-26 19:21:48,562 log artifact model at /User/mlrun/models/lgb-classifier.pkl, size: None, db: Y

[mlrun] 2020-01-26 19:21:48,603 run executed, status=completed
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...4c06e7,0,Jan 26 19:20:08,completed,sklearn-classifier,host=train-7mmh2kind=jobowner=admin,,SKClassifier=lightgbm.sklearn.LGBMClassifiercallbacks=[]key=modelname=lgb-classifier.pkltarget_path=/User/mlrun/modelsverbose=Falsextrain=/User/mlrun/models/xtrain.pqtxvalid=/User/mlrun/models/xvalid.pqtytrain=/User/mlrun/models/ytrain.pqtyvalid=/User/mlrun/models/yvalid.pqt,train_accuracy=0.732269862931968,training-validation-plot.htmlmodel


to track results use .show() or .logs() or in CLI: 
!mlrun get run 984d1c76d3744b8593395d4cec4c06e7  , !mlrun logs 984d1c76d3744b8593395d4cec4c06e7 
[mlrun] 2020-01-26 19:21:52,483 run executed, status=completed


In [9]:
tsk.outputs

{'train_accuracy': 0.732269862931968,
 'training-validation-plot.html': 'training-validation-plot.html',
 'model': '/User/mlrun/models/lgb-classifier.pkl'}